## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

# Connect to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

Mounted at /content/drive
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=UJo6029-8qIcZ5CWYnsE8OqCOOJAywUiSC-jN2O4Guw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh_5kJMewmn3GWawB5IuVnIPxlJFUNMSO6GVfmKCma8Famzj8CpFI4

Successfully saved authorization token.


In [3]:
import os
from glob import glob
!pip install shapely
!pip install geopandas
!pip install xarray
!pip install rioxarray
!pip install wxee
!pip install geojson
!pip install geemap

import geemap
import geojson
import shapely as shp
import geopandas as gpd
#import wxee
import numpy as np
import matplotlib.pyplot as plt


## User-defined Functions

In [8]:
# Function to Mask Clouds
def maskCloud(image):
  cloudMask = image.select(['pixel_qa']).bitwiseAnd(1<<5).eq(0)
  cloudShadowMask = image.select(['pixel_qa']).bitwiseAnd(1<<3).eq(0)
  final_mask = cloudMask.And(cloudShadowMask)
  return image.updateMask(final_mask)

In [21]:
# Function to add water Index bands
def addIndex(image):
  ndwi = image.normalizedDifference(['green', 'nir']).rename(['ndwi'])
  # https://www.tandfonline.com/doi/abs/10.1080/01431169608948714
  
  mndwi = image.normalizedDifference(['green', 'swir1']).rename(['mndwi'])
  # https://www.tandfonline.com/doi/abs/10.1080/01431160600589179
  
  awei = image.expression("blue + 2.5*green - 1.5*(nir+swir1) - 0.25*swir2", {
    'blue': image.select(['blue']),
    'green': image.select(['green']),
    'nir': image.select(['nir']),
    'swir1': image.select(['swir1']),
    'swir2': image.select(['swir2'])
  }).rename(['awei'])
  # https://www.sciencedirect.com/science/article/pii/S0034425713002873
  
  wiFi = image.expression("1.7204 + 171*green +3*red - 70*nir - 45*swir1 - 71*swir2",{
    'green': image.select(['green']),
    'red': image.select(['red']),
    'nir': image.select(['nir']),
    'swir1': image.select(['swir1']),
    'swir2': image.select(['swir2'])
  }).rename(['wiFi'])
  # https://www.sciencedirect.com/science/article/abs/pii/S0034425715302753
  
  return image.addBands([ndwi, mndwi, awei, wiFi])


In [77]:
# Function of ostu's method
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)

    indices = ee.List.sequence(1, size)

    def iFunc(i):
      aCounts = counts.slice(0, 0, i) 
      aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
      aMeans = means.slice(0, 0, i)
      aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(),[0]).get([0]).divide(aCount)
      bCount = total.subtract(aCount)
      bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
      return aCount.multiply(aMean.subtract(mean).pow(2)) \
            .add(bCount.multiply(bMean.subtract(mean).pow(2)))

    # Compute between sum of squares, where each mean partitions the data
    bss = indices.map(iFunc)

    return means.sort(bss).get([-1])

In [85]:
# Function of Otsu's Method
def thresholding(image):
  
  #waterIndex = index
  bands = ['ndwi', 'mndwi', 'awei', 'wiFi']

  #Compute the histogram of the NIR band. (the mean and variance are only FYI)
  hist_ndwi = image.select([bands[0]]).reduceRegion(
      reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
      geometry = roi.getInfo(),
      scale = 30,
      maxPixels = 10e12,
      bestEffort = True)
  
  thrh_ndwi = otsu(hist_ndwi.get(bands[0]+'_histogram'))

  watermask_ndwi = image.select([bands[0]]).gt(thrh_ndwi).rename(['watermask_ndwi']).selfMask()
  
  #water = image.select([index]).lt(threshold)
  return image.addBands([watermask_ndwi]) \
              .clip(roi) \
              .copyProperties(image) \
              .set('system:time_start', ee.Date.fromYMD(image.get('year'), 12, 31))

## Prepare ROI (Asia) for processing

### Load one-country parcels

In [4]:
# Load all parcels
os.chdir('/content/drive/MyDrive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Coastline_parcels_polygon')
path_parcels = glob(os.path.join(os.getcwd(), '*.geojson'))[0]
parcels = gpd.read_file(path_parcels)

In [5]:
#parcels.to_file('parcels.geojson', driver='GeoJSON')
with open('parcels.geojson') as f:
  parcels_geojson = geojson.load(f)

In [ ]:
#len(parcels_geojson['features'])

266

In [ ]:
# List all countries
#countries = sorted(list(set(parcels['NAME_0'])))
#print(countries)

['Bangladesh', 'Cambodia', 'China', 'India', 'Indonesia', 'Iran', 'Japan', 'Malaysia', 'Myanmar', 'North Korea', 'Pakistan', 'Philippines', 'South Korea', 'Sri Lanka', 'Taiwan', 'Thailand', 'Vietnam']


In [ ]:
# Subset Parcels to one Country
#parcels_one_country = parcels[parcels['NAME_0']==countries[0]]
#parcels_one_country

,GID_0,NAME_0,geometry
208,BGD,Bangladesh,"MULTIPOLYGON (((93.22541 23.10278, 93.12052 22..."
209,BGD,Bangladesh,"MULTIPOLYGON (((93.23961 23.12663, 93.22541 23..."
252,BGD,Bangladesh,"MULTIPOLYGON (((89.31118 20.35796, 89.07910 21..."


## GEE: export data coverage /asia

In [69]:
ROI = ee.FeatureCollection(parcels_geojson)

In [23]:
# Load Landsat Archives
ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterDate('2013', '2020') \
        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
        .filterBounds(ROI) \
        .sort("system:time_start", True)
ls7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
      .filterDate('1999', '2020') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(ROI) \
      .sort("system:time_start", True)
ls5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
      .filterDate('1984', '2013') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(ROI) \
      .sort("system:time_start", True)
ls4 = ee.ImageCollection("LANDSAT/LT04/C01/T1_SR") \
      .filterDate('1984', '1994') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(ROI)\
      .sort("system:time_start", True)

# Merge Collections + Mask Cloud + Add water index bands
ls_all = ls8.merge(ls7).merge(ls5).merge(ls4)
ls_all = ls_all.sort('system:time_start', True).map(maskCloud).map(addIndex)

In [35]:
# Count Observations at pixel level by year 
years = ee.List.sequence(1984, 2019)
counts = ee.ImageCollection.fromImages(
  years.map(lambda YEAR: 
            ls_all.select(['wiFi']).filter(ee.Filter.calendarRange(YEAR,YEAR,'year')).count().clip(ROI).set({'year':YEAR})))

In [63]:
#Map = geemap.Map(center=(20, 110), zoom=3)
#Map.addLayer(ROI, {}, 'ROI')

#vis = {'palette':["#ebedef", "#f6ddcc", "#f8c471", "#f4d03f", "#58d68d", \
#                "#27ae60", "#1abc9c", "#3498db", "#2471a3", "#6c3483"], 'min':0, 'max':29}
#Map.addLayer(counts.filter(ee.Filter.eq('year',1987)), vis, '1987')
#Map.addLayer(counts.filter(ee.Filter.eq('year',2019)), vis, '2019')
#Map

In [70]:
#img_count = counts.toBands()
#task_config = {
#    'description': 'data_coverage_asia_1km',
#    'fileNamePrefix': 'data_coverage_asia_1km',
#    'crs': 'EPSG:4326',
#    'scale': 1000,  
#    'region': ROI.geometry(),#.getInfo()['coordinates'],
#    'folder': 'myExportImage',
#    'skipEmptyTiles': True,
#    'fileFormat': 'GeoTIFF',
#    'maxPixels': 10e12
#    }
#task = ee.batch.Export.image.toDrive(img_count, **task_config)
#task.start() 
#task2 = ee.batch.Export.image.toDrive(img_count, **task_config)
#task2.start() 


In [95]:
print(task.status())
print(task2.status())

{'state': 'COMPLETED', 'description': 'data_coverage_asia', 'creation_timestamp_ms': 1638907056438, 'update_timestamp_ms': 1638909341789, 'start_timestamp_ms': 1638907140885, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_', 'https://drive.google.com/#folders/1F_SbqqdKupHKA64wB8jDTxegr-W-XyA_'], 'attempt': 1, 'id': 'NMMRUPAMGSTIMES332YBSKW3', 'name': 'project

## GEE: export water masks & data coverage /parcel

In [61]:
# Define roi
roi = ee.Geometry(parcels_geojson['features'][0]['geometry'])
roi.getInfo()

{'coordinates': [[[147.70812, 43.079548],
   [146.191735, 45.553622],
   [144.736087, 44.867934],
   [146.24364, 42.476689],
   [147.70812, 43.079548]]],
 'type': 'Polygon'}

In [64]:
# Load Landsat Archives
ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterDate('2013', '2020') \
        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
        .filterBounds(roi) \
        .sort("system:time_start", True)
ls7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
      .filterDate('1999', '2020') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(roi) \
      .sort("system:time_start", True)
ls5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
      .filterDate('1984', '2013') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(roi) \
      .sort("system:time_start", True)
ls4 = ee.ImageCollection("LANDSAT/LT04/C01/T1_SR") \
      .filterDate('1984', '1994') \
      .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']) \
      .filterBounds(roi)\
      .sort("system:time_start", True)

# Merge Collections + Mask Cloud + Add water index bands
ls_all = ls8.merge(ls7).merge(ls5).merge(ls4)
ls_all = ls_all.sort('system:time_start', True).map(maskCloud).map(addIndex)

In [65]:
# Temporally reduced at scale of one year
years = ee.List.sequence(1984, 2019)
ls_Treduced = ee.ImageCollection.fromImages(
    years.map(lambda YEAR: ls_all.filter(ee.Filter.calendarRange(YEAR, YEAR, 'year')) \
                          .median() \
                          .clip(roi) \
                          .set({'year': YEAR, 'system:time_start': ee.Date.fromYMD(YEAR, 12, 31)})))

In [66]:
# Filter out years where no data are available
# Add "bandlength" as new property
ls_filtered = ls_Treduced.map(lambda image: image.set('bandlength', image.bandNames().size()))
# Filter out the years where no data are available
ls_filtered = ls_filtered.filterMetadata('bandlength', 'not_equals', 0)

In [97]:
# Apply Otsu

#index = 'nir'
ls_wmsk = ls_filtered.map(thresholding)
ls_wmsk = ls_wmsk.select(['watermask_ndwi'])
print(ls_wmsk.size().getInfo())

36


In [96]:
Map = geemap.Map(center=(20, 110), zoom=3)
Map.addLayer(roi, {}, 'roi', False)
Map.addLayer(ls_filtered.filter(ee.Filter.eq('year', 2019)), {bands:['nir', 'red', 'green'], min:0, max:2550}, 'FCC')
Map.addLayer(ls_wmsk.select(['watermask_ndwi']).filter(ee.Filter.eq('year', 2019)), {'palette':'blue'}, 'water mask ndwi')
Map

Map(center=[20, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…